In [139]:
import urllib.request
from bs4 import BeautifulSoup
import re
import pandas as pd
from selenium import webdriver
import numpy as np

In [140]:
#WINIMAX
urlWin=r"https://www.winamax.fr/paris-sportifs/sports/1"
driver=webdriver.Chrome("chromedriver.exe")
driver.get(urlWin)


equipes=[eq.text for eq in driver.find_elements_by_class_name("sc-prPLn")]

cotes=[re.findall("^\d,?\d?\d?",cote.text) for cote in driver.find_elements_by_class_name("sc-fznWqX")]
cotes=np.array([cote[0].replace(",",".") for cote in cotes if len(cote)>0])
cotes=cotes.reshape(3,int(len(cotes)/3),order='F').astype(np.float)

heures=[eq.text for eq in driver.find_elements_by_class_name("sc-qQlyE")]
dfWin=pd.DataFrame({"equipe":equipes,"W1":cotes[0],"T":cotes[1],"W2":cotes[2]})
#dfWin["surebet"]=1/dfWin["W1"]+1/dfWin["T"]+1/dfWin["W2"]

dfWin=dfWin[~dfWin["equipe"].str.contains("LIVE")]

dfWin["heure"]=heures

In [141]:
dfWin.head()

,equipe,W1,T,W2,heure
2,Italie U21 - Suède U21,2.75,3.4,2.20,17:30
3,Luxembourg U21 - Irlande U21,15.00,6.5,1.13,17:30
4,Arménie - Macédoine du Nord,3.00,3.2,2.35,18:00
5,Géorgie - Estonie,1.55,3.9,6.25,18:00
6,Aucas - Independiente Del Valle,2.60,3.5,2.05,18:15


In [142]:
#Unibet
driver=webdriver.Chrome("chromedriver.exe")
urlUni=r"https://www.unibet.fr/sport/football"
driver.get(urlUni)


In [144]:
bettingBoxJour=driver.find_element_by_class_name("bettingbox-content")

equipes=bettingBoxJour.find_elements_by_class_name("cell-event")
equipes=[equipe.text for equipe in equipes]

cotes=np.array([cote.text for cote in bettingBoxJour.find_elements_by_class_name("price")])
cotes=cotes.reshape(3,int(len(cotes)/3),order='F').astype(np.float)

heures=[heure.text for heure in bettingBoxJour.find_elements_by_class_name("datetime")]

dfUni=pd.DataFrame({"equipe":equipes,"W1":cotes[0],"T":cotes[1],"W2":cotes[2],"heure":heures})

In [ ]:
dfUni.head()

In [147]:
dfMerge=dfUni.merge(dfWin,how="outer",on="equipe")
dfMerge["surebet"]=1/dfMerge[["W1_x","W1_y"]].max(axis=1)+1/dfMerge[["W2_x","W2_y"]].max(axis=1)+1/dfMerge[["T_x","T_y"]].max(axis=1)
dfMerge.sort_values("surebet")


,equipe,W1_x,T_x,W2_x,heure_x,W1_y,T_y,W2_y,heure_y,surebet
10,Bosnie-Herzégovine - Italie,9.00,5.25,1.28,20:45,10.00,5.25,1.33,20:45,1.042356
9,Belgique - Danemark,1.64,3.60,5.15,20:45,1.70,3.75,5.25,20:45,1.045378
7,Angleterre - Islande,1.14,7.00,16.50,20:45,1.15,8.00,18.00,20:45,1.050121
18,Pologne - Pays-Bas,3.80,3.60,1.85,20:45,3.80,3.80,1.90,20:45,1.052632
30,République Tchèque - Slovaquie,NaN,NaN,NaN,NaN,1.45,4.50,7.00,20:45,1.054735
3,Géorgie - Estonie,1.53,3.72,6.10,18:00,1.55,3.90,6.25,18:00,1.061572
12,Hongrie - Turquie,3.35,3.25,2.10,20:45,3.40,3.30,2.15,20:45,1.062264
8,Autriche - Norvège,1.33,4.75,8.35,20:45,1.30,5.00,9.00,20:45,1.062991
16,Kosovo - Moldavie,1.45,3.85,7.35,20:45,1.48,3.90,7.50,20:45,1.065419
17,Pays de Galles - Finlande,2.00,3.00,4.00,20:45,2.05,3.05,4.00,20:45,1.065674
